# PC lab: Convolutional Neural Networks 
----

<img src="https://raw.githubusercontent.com/flacout/deep-dream-demo/master/image/deep-dream.jpg" style="width:100%">

## Introduction

Convolutional neural networks caused a major step forward in the performance of image recognition. These networks are mostly identical to standard neural networks, in which features are first learned through multiple (layers of) convolutions. Obtained features are subsequently used as the input for a standard neural network, often performing a classification problem.

### Convolution

A convolution is the iteration of a kernel with size $ M \times N $ over a given input $ \textbf{X} $, performing a 2D linear combination of the weights $ W $  of the kernel with the overlapping area of the input. For a normal convolution with single striding and no padding, the output $ y_{ij} $ is equal to:

$$ y_{ij} = \sum_{a=0}^{m-1} \sum_{b=0}^{n-1} W_{ab} x_{(i+a)(j+b)} $$

During a convolution, the kernels slides over the input image to obtain a new image of outputs. The stride of a kernel defines the horizontal and vertical stepsize during iteration. Input data can be padded with multiple layers of a zero-filled border, increasing the output dimensions.

It is important to understand that a convolution applies the same operation at every local patch in the input. In this sense, convolutions are useful when you expect the input data to contain regularly appearing **local patterns**.


**convolution step with M,N = 3; stride = 1 and padding of 1**
<img src="https://miro.medium.com/max/790/1*1okwhewf5KCtIPaFib4XaA.gif">

**several other examples. An extended explanation on all types of convolutions can be found [here](https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d)**

## Convolutional neural network

A convolutional neural network usually processes the image with **multiple sequential convolutions**. For each layer, the kernel is evaluated for **all channels** of the input data. It is important to understand that the output depth is correlated to the amount of different kernels, or features, every node has been initialized with. The kernel, although often depicted as only evaluating one layer, actually takes **the sum of all layers (channels)** to obtain an output.

<img src="https://miro.medium.com/max/2560/1*ciDgQEjViWLnCbmX-EeSrA.gif" style="width:100%">

<img src="https://miro.medium.com/max/2510/1*vkQ0hXDaQv57sALXAJquxA.jpeg" style="width:100%">

A classic example of a convolutional neural network applies an activation function (e.g. **ReLU**) on the output of every convolutional layer, after which the activation signals are **maximum pooled**. Maximum pooling reduces the dimensionality of the input, which can be used to reduce the amount of parameters present in a neural network, which in turn reduces overfitting and computational burden. Maximum pooling is also initialized with specific arguments such as kernel size, stride and padding.

<img src="https://computersciencewiki.org/images/8/8a/MaxpoolSample2.png">

The last layers of every convolutional neural network always consist of several fully connected layers. The mathematical description of these layers were discussed in the previous PC lab. One can interpret the convolutional layers as the section of the network in which local patterns are extracted (edges, contours, contrasts,...). These are used as inputs for the fully connected neural netwerk, which combines these features to train the classifier. 

The following picture shows a visualization of what a CNN extracts at each layer, starting from the first layers on the left going deeper towards the right. This visualization is obtained by optimizing an input image to maximally activate the convolution filters.

<img src='https://1.bp.blogspot.com/-icbxyuiDoA0/WgEivsyFIgI/AAAAAAAACKo/jsfMgFlfiVA233zXg8xAH3ZAKOchgLb-wCLcBGAs/s1600/image4.png'>

**The softmax function** takes the n-dimensional output of the model and rescales these values to probabilities that sum up to one. It is typically used as the output layer for multiclass classification.

$$ \sigma(\hat{y})_j = \frac{e^{\hat{y}_j}}{\sum_{k=1}^{K} e^{\hat{y}_k}} $$

# Structure of the exercises.

During this PC-lab you will be introduced with an examplory workflow when training a predictive model for image recognition. More specifically, we will create a <b>convolutional neural network</b> to <b>recognize vehicles from animals</b> using the <b>CIFAR-10</b> dataset in <b>PyTorch</b>.

Let's first recap from last week what is needed to train a model in PyTorch:

1. Define a model (class) with \__init__(self, ...) and forward(self, ...) functions
2. Instantiate the model
3. Insantiate a [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions)
4. Instantiate an [optimizer object](https://pytorch.org/docs/stable/optim.html), to which you pass the parameters you want to optimize
5. Wrap your data splits in a [data loader](https://pytorch.org/docs/stable/data.html)
6. Perform an epoch, consisting of:
    - Iterating through all training data and
        - resetting gradients
        - forward pass
        - compute loss
        - backward pass
        - update parameters
    - Iterating through all validation data and
        - forward pass
        - compute loss

We iterate through validation data at every epoch so we can keep track of the performance throughout training.

Last week, we used simple, rudimentary code in order to perform the training steps. This week, we will provide you with code that abstracts away the repetitive steps of this process, allowing you to focus on the fun stuff: implementing models and watching them train.




<img src="https://www.researchgate.net/profile/Jean-Elsner/publication/329969195/figure/fig1/AS:708799606317059@1546002403551/Images-from-the-CIFAR-10-13-dataset-and-their-corresponding-classes-CIFAR-10.ppm" width='25%' align="right">

## CIFAR-10 Data loading and preprocessing

- 60000 $32 \times 32$ colour images
- 50000 training images + 10000 validation images
- 10 classes $\rightarrow$ 6000 images per class
- classes are: airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks


Collected for **MSc thesis** 

<ul>
<a href="learning-features-2009-TR.pdf">Learning Multiple Layers of Features from Tiny Images</a>, Alex Krizhevsky, 2009.
</ul>

A list featuring some papers using CIFAR-10 can be found [here](http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#CIFAR-10). To load the CIFAR-10 dataset into our environment, we can use the pytorch module. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

validationset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

validation_loader = torch.utils.data.DataLoader(validationset, batch_size=32,
                                         shuffle=False, num_workers=2)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Building the network


<div class="alert alert-success">
    <h2>Exercise:</h2>
        <p>Look up the documentation for the (<a href=https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html>2-dimensional convolutional layer</a>). What are the parameters that need to be defined? What kind of dimensions will the input have for different types of parameters? Use the code-block below to experiment, try to get an understanding of how you should define the Conv layer and what it does to your input dimensionality. Do the same experiment for the <a href=https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html>Max Pooling layer</a>. Finally, carefully look at the note at the bottom of the code block.</p>
        </div>

In [ ]:
example_batch = next(iter(train_loader))
X_example, y_example = example_batch
print(X_example.size(), y_example.size())


test_conv = nn.Conv2d(...)
test_pool = nn.MaxPool2d(...)

X_after_conv = test_conv(X_example)
print(X_after_conv.size())

X_after_pool = test_pool(X_example)
print(X_after_pool.size())

X_after_both = test_pool(test_conv(X_example))
print(X_after_both.size())

# NOTE: a quick way to chain multiple 'nn.'-type layers is through the use of nn.Sequential():
# e.g. ConvPoolBlock = nn.Sequential(nn.Conv2d(...), nn.ReLU(), nn.MaxPool2d(...))
# x = ConvPoolBlock(x) can now be used to sequentially execute the conv, relu and maxpool in one operation.


<div class="alert alert-success">
    <h2>Exercise:</h2>
        <p>Implement a convolutional neural network. The network should have 2 or 3 convolutional layers (<a href=https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html>Conv2d</a>). Use 3x3 or 5x5 kernel sizes. Use Max Pooling  layers with 2x2 kernels to reduce dimensionality after convolutional layers. You are free in choosing the rest of the hyperparameters. At the end of the network, the output should be flattened (<code>x.view(-1, ...)</code> in forward pass) and sent through some <a href=https://pytorch.org/docs/stable/generated/torch.nn.Linear.html?highlight=linear#torch.nn.Linear>fully connected layers</a>. The final output of the model should be of size <code>batch x 10</code>, representing the output score for the ten possible classes. The difficulty here is figuring out how much dimensions you have as input to the first fully connected layer (right after flattening). You can use the previous code block in order to help with this.</p>
</div>


Keep in mind that the larger the model (both in number of layers and size of every layer), the longer training time.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Use this Sequential as blueprint for other layers, remember to fill in the parameters.
        self.ConvBlock1 = nn.Sequential(nn.Conv2d(3, ...), nn.ReLU(), nn.MaxPool2d(...))
        self.ConvBlock2 = ...
        
        ... # more if you want

        self.FC_layers = nn.Sequential(nn.Linear(..., ...), nn.ReLU(),
                                       ... , # more if you want
                                       nn.Linear(..., 10))

    def forward(self, x):

        x = self.ConvBlock1(x)
        x = self.ConvBlock2(x)
        ... # more if you want

        # Flatten array, '.view(...)' works similarly to numpy's '.reshape(...)'
        # Fill in the number of dimensions that your first linear layer expects as input.
        x = x.view(-1, ...)
        x = self.FC_layers(x)
        return x

Notice the similarities between the creation of this neural network and the one created in the previous lab. To be able to call a variant of this network, we can choose to add parameter values to `def __init__(self, .....)`, such as kernel sizes, or the total amount of nodes in each fully connected layer.

## 3. Training the model

### Helper functions


The next step is to train the model. The train_loader and validation_loader object is used to obtain data batches of fixed size. The `fit()` function trains the model for a specified amount of epochs while storing data obtained during training with the `logger()` class.

In [ ]:

def fit(model, train_loader, validation_loader, criterion, optimizer, log, epochs=20, device='cpu'):
    epoch = 0   # set starting epoch
    model = model.to(device)

    while epoch<epochs:
        print("\nepoch {}".format(epoch))
        epoch +=1
        model.train()
        for i, data in enumerate(train_loader):   # iterate randomized batches
            optimizer.zero_grad()
            X_batch, y_batch = data
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            y_hat = model.forward(X_batch)
            loss = criterion(y_hat, y_batch)
            loss.backward()   # Calculate gradient
            optimizer.step()   # Update weights using defined optimizer
            if device != 'cpu':
                log.log_metrics(y_batch.data.cpu().numpy(), y_hat.data.cpu().numpy(), loss.item())
            else:
                log.log_metrics(y_batch.data.numpy(), y_hat.data.numpy(), loss.item())
            if (i%100 == 1):
                log.output_metrics()
        
        # Repeat this process for the validation dataset
        model.eval()
        for i, data in enumerate(validation_loader, 0):
            X_batch, y_batch = data
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            y_hat = model.forward(X_batch)
            loss = criterion(y_hat, y_batch)
            if device != 'cpu':
                log.log_metrics(y_batch.data.cpu().numpy(), y_hat.data.cpu().numpy(), loss.item(), validation=True)
            else:
                log.log_metrics(y_batch.data.numpy(), y_hat.data.numpy(), loss.item(),validation=True)
        log.output_metrics(validation=True)
        
        

<div class="alert alert-warning">
    <h2><code>class logger()</code> </h2>
    <p><code>logger()</code> has been implemented as a convenient way to store model metrics throughout the training process.  An object of this class can be created before training and is used for calculating, storing, printing and plotting model metrics. The object does only store the metrics as defined at initialization. Feel free to add yor own metrics to the class.</p>
</div>

In [ ]:
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline 

class logger(object):
    def __init__(self, metrics, max_i):
        self.i = [0,0]
        self.max_i = max_i
        self.log_loss, self.log_auc, self.log_acc = False, False, False
        self.metrics = {"train":{}, "validation":{}}
        if "loss" in metrics:
            self.log_loss = True 
            self.metrics["train"].update({"loss":[0]})
            self.metrics["validation"].update({"loss":[0]})
        if "acc" in metrics:
            self.log_acc = True
            self.metrics["train"].update({"acc":[0]})
            self.metrics["validation"].update({"acc":[0]})
        
    def log_metrics(self, y_true, y_hat, loss, validation=False):
        if validation:
            sw = 1
            sw_str = "validation"
        else:
            sw = 0
            sw_str = "train"
        self.i[sw] += 1
            
        if self.log_loss:
            update = (self.metrics[sw_str]["loss"][-1]*(self.i[sw]-1)+loss)/self.i[sw]
            self.metrics[sw_str]["loss"].append(update)
        if self.log_acc:
            acc = sum(y_hat.argmax(axis=1) == y_true)/len(y_true)
            update = (self.metrics[sw_str]["acc"][-1]*(self.i[sw]-1)+acc)/self.i[sw]
            self.metrics[sw_str]["acc"].append(update)
        
    def output_metrics(self, validation=False):
        data = "validation" if validation else "train"
        if validation:
            print_str = "\n{:<10s}:\t100.0%".format(data)
        else:
            print_str = "\r{:<10s}:".format(data)
            print_str += "\t{:4.2f}%".format((self.i[0]%self.max_i)/self.max_i*100)
        for k, v in self.metrics[data].items():
            print_str += "\t{}: {:5.3f}".format(k, v[-1])
        print(print_str, end = "")
                
    def plot_metrics(self):
        fig, axes = plt.subplots(len(self.metrics["train"]),
                               2, figsize=(12,6*len(self.metrics["train"])))
        for i, dict_0 in enumerate(self.metrics.items()):
            for j, dict_1 in enumerate(dict_0[1].items()):
                axes[j,i].plot(range(len(dict_1[1][1:])),dict_1[1][1:])
                axes[j,i].set_title("{} {}".format(dict_0[0], dict_1[0]))


### Training

Now that we have loaded our data, defined our convolutional neural network, and created all the functions necessary for training, the actual training process can start. For this instance, we will use the cross entropy loss to optimize our model. Notice how `nn.CrossEntropyLoss()` incorporaties the softmax function on the inputs. [Adam](http://sebastianruder.com/optimizing-gradient-descent/index.html#adam) is used to determine the step size using the gradient of the loss with respect to the weights. Adam is currently often considered the best option for this task.

In [ ]:
import torch.optim as optim
from sklearn.metrics import confusion_matrix

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
log = logger(metrics=["loss","acc"], max_i =len(train_loader))
fit(model, train_loader, validation_loader, criterion, optimizer, log, epochs=20, device='cuda')

Is this performance in line with your expectations? Think about how the input dimensionality (number of pixels in every image) impacts performance and runtime.

## 4. Evaluating the model

### Metrics

After training we can plot saved metrics using the `logger.plot_metrics()` function. 

In [ ]:
log.plot_metrics()

<div class="alert alert-success">
    <h2>Exercise:</h2>
        <p><b>Evaluate</b> the metrics after 15-20 epochs. Should the model train longer?</p>
</div>

<div class="alert alert-success">
    <h2>Exercise:</h2>
        <p><b>Try</b> different optimizer functions and compare the different loss curves. Additionally, <b>try</b> dropout after the linear layers, compare training dynamics.</p>
</div>

### Confusion matrix

<div class="alert alert-success">
    <h2>Exercise:</h2>
        <p><b>Write out </b> some code to obtain the confusion matrix of the predictions on the validation data. (Second code block under this)</p>
</div>

In [ ]:
model.eval()
device = model.ConvBlock1[0].weight.device # were you working on cpu or cuda?

y_hat_all = []
y_true_all = []

for i, data in enumerate(validation_loader):
    X_batch, y_batch = data
    X_batch = X_batch.to(device)
    y_hat = model.forward(X_batch)
    if device != 'cpu':
        y_hat_all.append(y_hat.cpu())
    else:
        y_hat_all.append(y_hat)
    y_true_all.append(y_batch)
y_hat_all = torch.cat(y_hat_all)
y_true_all = torch.cat(y_true_all)
print(y_hat_all[:5])
print(y_true_all[:5])

In [ ]:

from sklearn.metrics import confusion_matrix

def confusion_matrix_from_variables(y_hat, y_true):
    ... # convert y_hat to a format that works for confusion_matrix
    matrix = confusion_matrix(y_true, y_hat)
    return matrix


In [ ]:
confusion_matrix_from_variables(y_hat_all, y_true_all)

### Sample predictions

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    fig, ax = plt.subplots(1,1,figsize=(20,20))
    plt.imshow(np.transpose(npimg, (1, 2, 0)),)

In [ ]:
# creating a shuffled version of the validation loader. Now, every time you run this cell you will get different images
validation_loader = torch.utils.data.DataLoader(validationset, batch_size=32,
                                         shuffle=True, num_workers=2)

dataiter = iter(validation_loader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images[:8]),)
outputs = model.forward(images.to(device))
predicted = torch.max(outputs.data, 1)[1]
print('\t'.join(np.array(classes)[predicted.cpu().numpy()[:8]]))

<div class="alert alert-success">
    <h2>EXTRA (optional) Exercise:</h2>
        <p><b>Optimize</b> the model in any way you can. You can change to the architecture of the model, vary the hyperparameters and add any of the many optimization techniques found in literature. </p>
</div>